# A Python implementation of Simplicial Sets

The general goal in topological data analysis (TDA) is to associate suitable topological invariants to data sets from which one can make meaningful inferenres about the data. 

The process involves replacing a data set with a suitable (filtered) topological space. Since the standard notion of a topological space is continous and infinite it is not suitable for computational purposes. One needs to a use a more combinatorial and finite representation of spaces in general. The notion of a simplicial complex is by far the simplest and most used contruction as a finite version of a space. 

The bulk of methods in TDA is to go from a data set to a simplicial complex. Since a data set is discrete it has no interesting topology. The standard way is to thicken it using a choice of a scale $\epsilon > 0$. The Cech nerve of the open covering thus obtained is the simplicial complex one works with. The are many alternative simplcial complexes one can use that approximates the Cech complex to make computations more feasable.

The aim of this note is to argue the notion of simplicial sets instead of simplicial complexes as a good alternative. A simplicial set is a generalization of a simplicial complex. It is more combinatorial and less geomertic that a simplicial complex. Simplicial sets have been long in use by homotopy theorists as a model for homotopy theory of topological spaces. The algebraic invariant used in TDA are invariants of the homotopy type of the uderling space/complex. Therefore it makes sense to use the correct model for homotopy theory. Besides, the the category of simplicial sets, unlike simplcial compleces, is closed under colmits, making it possible to take quotients. 

We define python classes for data structures for writing simplcial complexes, simplcial sets and delta complexes (an intermediate notion between the two)


## Simplicial complexes

A simplicial complex is a geometric figure obtained by gluing simplices of various dimensions along the faces. A simplex of dimesion $n$ is the covex hull of the set of standard basis bactors $\{e_1,\ldots, e_{n+1}\}$ in $\mathbb{R}^{n+1}$. Therefore a $0$-simplex is a point, a $1$-simplex is a line segment, a $2$-simplex is a triangle, a $3$-simplex is a tetrahedron and so on. 

In a simplcial complex every simplex is uniquely expressed by the collection of its vertices. For instance a vertex set $\sigma =\{0,\ldots,m\}$ forms the $m+1$ vertices of a $m$-simplex $\sigma$. The simplex $\sigma$ has $m+1$ faces denoted by $d_i(\sigma), 0\leq i \leq m$, which are $m$-simplices, associated with them. Here $$d_i(\sigma) = \{0,\ldots,\hat{i}, \ldots,m\}$$ obtained by omitting the $i$-th vertex. 

An abstract simplicial complex is a pair $X = (V,\Sigma)$ where $V$ is a finite set and $\Sigma \subset \mathcal{P}(V)$ is a collection of subsets of $V$ with the following property: If $\sigma \in V$ and $\tau \subset \Sigma$, then $\tau \in \Sigma$.

Given such a pair $X = (V,\Sigma)$ we form a simplicial complex as follows. The sets of vertices are in bijection with the set $V$. An ordering on $V$ gives and ordering on the vertex set. If $V = \{a_0,\ldots,a_n\}$, a subset $\sigma = \{a_{k_0} \ldots a_{k_n}\}$ (preserving the ordering of $V$) denotes the corresponding $n$-simplex. For example we may model a circle using the abstract simplcial complex with $V = \{1,2,3\}$ and $\Sigma = \{\{1,2\},\{2,3\}, \{1,3\},\{1\}, \{2\},\{3\}\}$.

Given an abstract simplicial complex $(V,\Sigma)$, let $\Sigma_k \subset \Sigma$ be the subset of $k$-simplices. There are faces maps $$d_i: \Sigma_k \to \Sigma_{k-1}, \, 0 \leq i \leq k$$ $$d_i(\{a_{l_0},\ldots,a_{l_n} \}) = \{a_{l_0},\ldots,\widehat{a_{l_i}}, \ldots, a_{l_n}\}$$

### Homology

Let $C_k = \mathbb{Z}(\Sigma_k)$ be the free abelian group generated by the set of $k$-simplices. There is a boundary maps of abelian groups $$\partial_k: C_k \to C_{k-1}$$ $$\partial_k = \sum_{i=0}^{n}(-1)^id_i$$ The boundary maps have the property that $\partial_{k-1}\circ \partial_k = 0$.

This forms a chain complex of abelian groups $C_{\bullet} = \{C_k\}, \, (k\geq 0)$ connected by boundary maps $\partial_k$. The $k$-th homology group of the simplicial complex is the $k$-homology of the chain complex, $$H_k^{\Delta}(X) = \mbox{Ker}(\partial_k)/\mbox{Im}(\partial_{k+1}).$$ Given an abelian group $A$, the simplicial homology groups with coefficients in $A$ is the homology of the chain complex $C_{\bullet}\otimes A$: $$H_k^{\Delta}(X;A) = H_k(C_{\bullet}\otimes A)$$

The Betti numbers are defined as $$\beta(X)_k = \dim _FH_k^{\Delta}(X;F)$$ wher $F$ is a field. The $k$-th Betti number for $k>1$ measures the number of $k$-dimensional holes in the topological realization of $X$. The zeroth betti number $\beta(X)_0$ measures the number of connected components. 

### The boundary matrix and Betti numbers

Since the groups $C_k(X)$ are equipped with bases $\Sigma_k$, the operators $\partial_k$ can be expressed as matrices $D(k)$ whose rows are indexed by $\Sigma_{k-1}$ and columns are indexed by $\Sigma_{k}$. 

For $\sigma \in \Sigma_{k}$ and $\tau \in \Sigma_{k-1}$ the entry $D(k)_{\sigma\tau} = 0$ if $\tau \not\subset \sigma$ and $=(-1)^i$ if $\tau \subset \sigma$ and $\tau$ is obtained from $\sigma$ by removing the $i$-th element of the subset $\sigma$.

$$\beta(X)_k = \dim_F\left( \frac{\mbox{Ker}\,D(k)}{\mbox{Im}\,D(k+1)} \right) = \mbox{len}(\Sigma_k) - \mbox{rank}(D(k)) - \mbox{rank}(D(k+1))$$

### Python implementation

An abstract simplicial complex can be encoded as a set of sets. In python we implement this as a list of lists.

For example one can implement a simplcial complex structure of a circle as follows:

$X$ = ["x","y","z", "xy", "yz", "xz"]

To every simplex in $X$ we can associate a simplex "tree" consisting of all its faces in lower dimensions. We use a list of lists to use a write a tree. The simplex tree associated with the simplex "xy" is ["xy", ["x",[]], ["y",[]]]. The simplex trees of all the simplices can be combined to form a $X.\mbox{dim}+1$-partitite graph (also written as a list of lists), called the facetree, encoding the face maps in its entirety. The facetree of $X$ is ["xy",["x",[]],"yz",["y",[],"z",[]], "xz",["x",[]],["z",[]]].
   

In [12]:
import numpy as np

In [13]:
#---flatten an iterated list to a simple list --------

def flatten(A):
	if A == []: return A
	if type(A[0]) == list:
		return flatten(A[0]) + flatten(A[1:])
	else: return [A[0]] + flatten(A[1:])


### The scomplex class

In [14]:

class scomplex():
	def __init__(self, simplices = []):
		self.simplices = list(set(flatten([self.simplexfaces(simplex) for simplex in simplices])))
		self.vertices = [simplex for simplex in self.simplices if len(simplex)-1 == 0]
		self.dim = max(len(x)-1 for x in self.simplices)
		self.topdimsimplices = [simplex for simplex in self.simplices if len(simplex)-1 == self.dim]
		self.topsimplices = self.topdimsimplices  
		self.facetree = self.facetree()

	def Sigma(self,k):
		return [simplex for simplex in self.simplices if len(simplex)-1 == k]
	
	def d(self,simplex,i):
		if i in range(len(simplex)):
			return simplex[0:i] + simplex[i+1:len(simplex)]
		else:pass

	def simplextree(self, simplex):
		simplextree = []
		if len(simplex) == 1: return [simplex,[]]
		else:
			simplextree = [simplex]
			for i in range(len(simplex)):
				simplextree.append(self.simplextree(self.d(simplex,i)))
		return simplextree
	

	def simplexfaces(self,simplex):
		return flatten(self.simplextree(simplex))


	def facetree(self):
		return [self.simplextree(simplex) for simplex in self.topsimplices]	

	def D(self,k):
		if k in range(self.dim +1):
			if k == 0: return np.zeros(len(self.Sigma(0)))
			else:
				n = len(self.Sigma(k))
				m = len(self.Sigma(k-1))
				A = np.zeros((m,n))
				for i in range(m):
					for j in range(n):
						for l in range(k+1):
							if self.Sigma(k-1)[i] == self.d(self.Sigma(k)[j],l):
								A[i,j] = (-1)**l
							else: pass
				return A
		elif k == self.dim+1: return np.zeros((1,1))	
		else: pass


	def betti(self):
		return [len(self.Sigma(k)) - np.linalg.matrix_rank(self.D(k)) - np.linalg.matrix_rank(self.D(k+1)) for k in range(self.dim +1)]


In the implementation it is not necessary to enter all the simplices. The scomplex class will automatically include all the lower dimensional faces of a simplex. The class scomplex has attributes simplices (lists down the simplices of all dimensions), Sigma($i$) = list of $i$ dimensional simplcies, and betti = list of betti numbers from dimesion $0$ to dimension of the complex.

The $0$-th Betti number is the number of connected components and the $i$-th Betti number (for $i>0$) is the number of $i$-dimensional holes in the topological realization of the simplicial complex.


In [15]:
Z = scomplex(["xy","yz","xz","zw","xyz","yzw","xzw","xyw"])
#Z is topologically a sphere

#print("simplices =", Z.simplices, "\n")
print("Sigma_0 = ", Z.Sigma(0),"\n")
print("Sigma_1 = ", Z.Sigma(1),"\n")
print("Sigma_2 = ", Z.Sigma(2),"\n")


print("Betti numbers =",Z.betti())

Sigma_0 =  ['y', 'x', 'z', 'w'] 

Sigma_1 =  ['yz', 'zw', 'xw', 'xz', 'yw', 'xy'] 

Sigma_2 =  ['yzw', 'xyw', 'xyz', 'xzw'] 

Betti numbers = [1, 0, 1]


## Point cloud data $\to$ simplicial complex

The standard route in TDA is to go from a data set $\mathbb{X}$ suitably expressed as a finite metric space (point cloud) and a choice of a resolution $\epsilon$ to a topological space. The homology/betti numbers of the space are the topological invariants of the data at the given resolution.

The topological space above is usually the realization of a simplicial complex $\mathcal{N}(\mathbb{X},\epsilon)$ arising from taking the Cech nerve associated with the covering of open balls of radius $\epsilon$ around the data points. The Cech nerve is often replaced by a subcomplex called the Rips complex $R(\mathbb{X},\epsilon)$.

The Rips complex has $\mathbb{X}$ as its vertex set. We declare $\sigma = \{x_0, \ldots, x_n\}$ to span a $n$-simplex of $R(\mathbb{X},\epsilon)$ iff $d(x_i,x_j) \leq \epsilon$ for all pairs $x_i,x_j \in \sigma$. 



## Delta Complexes (semi-simplicial sets)

A delta complex (or semi-simplcial sets) is a more flexible notion than a simplicial complex where simplices are no longer uniquely determined by their vertices. A vertex set $\{0,\ldots,m \}$ can for form the vertices of more than one $m+1$-simplex. A delta complex is defined as a collection of sets $K_n, \, n\geq 0$. We think of each $K_n$ as the set of $n$-simplices. For every $n$ there are face maps $d_j:K_n \to K_{n-1},\,\, 0\leq j \leq n$, satisfying the identity

\begin{equation}
d_id_j = d_{j-1}d_i \,\,(i < j)
\end{equation}

Simplicial homology and betti numbers are defined similarly. Define abelian groups of $k$-chains $C_k = \mathbb{Z}(K_n)$ and $$\partial_k: C_k \to C_{k-1}$$ $$\partial_k = \sum_{i=0}^{n}(-1)^id_i$$

### Torus

A torus can be obtained from $I \times I$ by identifying the sides in the way prescribed above. This can be given a delta complex structure as follows. 
$K_0 = \{x\}, K_1 = \{a,b,c\}, K_2 = \{A,B\}$

$d_0(a) = d_1(a) = x$, $d_0(b) = d_1(b) = x$, $d_0(c) = d_1(c) = x$

$d_0(A) = a, d_1(A) = b, d_2(A) = c$

$d_0(B) = a, d_1(B) = b, d_2(B) = c$

Notice that this is not a valid simplicial complex as the triangles $A$ and $B$ have one vertex $x$, which violatesthe rule that every $n$-simplex must contain $n+1$-vertices. The minimum triangulation of a torus in the simplicial complex setting requires atleast seven vertices.

### Python implementation 

The delta complex data structure consists of a collection of simplices (the sets $K_n$ in definition) implemented as a list of sets and a structure tree containing information about the all the face maps of all the simplices. For example the topological circle can be given a delta complex strucure as follows. 

$X = (\mbox{simplices}, \mbox{structuretree})$

$X.\mbox{simplices}= $  [{"x"},{"a"}]  
(one $0$-simplex "x" and one $1$-simplex "a")

$X.\mbox{structuretree} = $["a",["x",[]],["x",[]]]  
($d_0$("a") = $d_1$("a") = "x")

(Note that this is not a simplicial complex)

Delta complexes allows for very compact representations of space which would otherwise require a large number of simplices for a simplcial complex representation.

A delta complex structure on a torus can be implemented as:

$T.\mbox{simplices} = $ [{'x'},{'a','b','c'},{'A','B'}]

$T.\mbox{structuretree} = $ [['A',['a',['x',[]],['x',[]]],['b',['x',[]],['x',[]]],['c',['x',[]],['x',[]]]],['B', ['a',['x',[]],['x',[]]],['b',['x',[]],['x',[]]],['c',['x',[]],['x',[]]]]]

### The boundary matrix and Betti numbers

Using bases $K_n$ for $C_n$ the boundary operator $\partial_k:C_k \to C_{k-1}$ can be represented using a $\mbox{len}(K_{k-1}) \times \mbox{len}(K_k)$ matrix $D(k)$.  The matrix can be computed using the formula for the boundary map $\partial_k$. 

As in the simplcial complex case
$$\beta_k = \mbox{len}(K_k) - \mbox{rank}(D(k)) - \mbox{rank}(D(k+1))$$


In [16]:
def isnode(A,x):
	if x in flatten(A): return True
	else: return False


### The semi_sset class

In [17]:

class semi_sset():
	def __init__(self, simplices = [], structuretree = []):
		self.simplices = simplices
		self.structuretree = structuretree
		self.topdim = len(self.simplices)-1

	def simplexdim(self,simplex):
		return max(i for i in range(len(self.simplices)) if simplex in self.simplices[i])
		#return depth(self.simplextree(simplex))	

	def Sigma(self,k):
		return list(self.simplices[k])

	def codim1sub(self,i):
		X = semi_sset(self.simplices, self.structuretree[i])
		return X

	def simplexlocate(self,simplex):
		out= ""
		struct = self.structuretree
		if simplex in struct: return str(struct.index(simplex))+"-"
		else:
			for l in struct:
				if isinstance(l,list):
					if isnode(l,simplex):
						out += str(struct.index(l)) + str(self.codim1sub(struct.index(l)).simplexlocate(simplex))
		return out

	def addsimplex(self,name,simplexdim):
			while (simplexdim > len(self.simplices)-1):
				self.simplices.append(set())
		
			self.simplices[simplexdim].add(str(name))

	def simplextree(self,simplex):
		L = self.structuretree
		addresses = self.simplexlocate(simplex)
		firstaddress = addresses.split("0-")[0]
		for direction in firstaddress:
			L = L[int(direction)]
		return L

	
	def d(self,simplex,i):
		if i in range(self.simplexdim(simplex)+1):
			return self.simplextree(simplex)[i+1][0]
	
	def delta(self,simplex):
		n = len(self.Sigma(self.simplexdim(simplex)))
		m = len(self.Sigma(self.simplexdim(simplex)-1))
		out = np.zeros(m)
		
		for i in range(self.simplexdim(simplex)+1):
			out[self.Sigma(self.simplexdim(simplex)-1).index(self.d(simplex,i))] += (-1)**i
		return out

			
	def simplexfaces(self,simplex):
		return set(flatten(self.simplextree(simplex)))

	def simplexboundaries(self,simplex):
		L = []
		for k in range(1,len(self.simplextree(simplex))+1):
			L.append(self.simplextree(simplex)[k][0])
		return L

	def boundary_matrix(self,k):
		self.clean()
		if k in range(self.topdim +1):
			if k == 0: return np.zeros(len(self.Sigma(0)))
			else:
				n = len(self.Sigma(k))
				m = len(self.Sigma(k-1))
				A = np.zeros((m,n))
				for j in range(n):
					A[:,j] = self.delta(self.Sigma(k)[j])
				return A
		elif k == self.topdim+1: return np.zeros(1)	
		else: pass

	def clean(self):
		L = set(flatten(self.structuretree))
		for i in range(len(self.simplices)):
			for x in self.simplices[i]:
				if x not in L:
					self.simplices[i] = self.simplices[i] - {str(x)}
		
		return self	
    
	def betti(self):
		self.clean()

		return [len(self.Sigma(k)) - np.linalg.matrix_rank(self.boundary_matrix(k)) - np.linalg.matrix_rank(self.boundary_matrix(k+1)) for k in range(self.topdim +1)]


### Test run

In [18]:
X = semi_sset([{"x","y","z"},{"xy","yz","xz"}], [["xy",["x",[]],["y",[]]],["yz",["y",[]],["z",[]]],["xz",["x",[]],["z",[]]]])
#X is a circle
print(X.betti())

T = semi_sset([{'x'},{'a','b','c'},{'A','B'}],[['A',['a',['x',[]],['x',[]]],['b',['x',[]],['x',[]]],['c',['x',[]],['x',[]]]],['B', ['a',['x',[]],['x',[]]],['b',['x',[]],['x',[]]],['c',['x',[]],['x',[]]]]])
#T is a torus
print(T.betti())



[1, 1]
[1, 2, 1]


## Simplicial Sets

A simplicial set is a generalization of a delta complex which has degenerate simplices along with the usual simplices in various dimensions. A simplicial set $X$ consists of sets of $n$-simplices $X_n$ for $n \geq 0$. There are face maps and degeneracy maps $$d_i : X_n \to X_{n-1}, \,\, (0\leq i \leq n)$$ $$s_i: X_n \to X_{n+1},\,\, (0\leq i \leq n)$$ A simplex in the image of a degeneracy is a degenerate simplex. The face and degeneracy maps satisfy the following identities

\begin{equation}
d_id_j = d_{j-1}d_i \,\, (i < j)\\
d_is_j = s_{j-1}d_i \,\, (i < j)\\
d_is_j  = \mbox{id}\,\, (i=j,j+1)\\
d_is_j  = s_jd_{i-1}\,\, (i > j)\\
s_is_j = s_js_{i-1}, (i > j)\\
\end{equation}

A simplicial set has degenerate simplices in all (infinitely many) dimensions. However in our implementation we truncate at the topdimension. 

### Simplicial complex to simplicial set

A simplicial complex is naturally a simplicial set. Given a simplicial complex $S = \Sigma_0, \Sigma_1, \ldots$, we define a simplicial set $X$ with $$X_0 = \Sigma_0$$ and $$X_n = \Sigma_n \cup \left(\bigcup_{i=0}^n s_i(X_{n-1})\right)$$ the face map $d_i$ acts on a simplex by removing the $i$-the vertex and the degeneracy map $s_i$ acts by repeating the $i$-th vertex.

## Quotients

### Example
The real advantage of simplicial sets over simplicial complexes or delta complexes is that the category of simplicial sets is closed under colimits, i.e. one can do quotient constructions on simplicial sets. 

For example, a sphere is the identification space of a triangle with its boundary identified to a point. There is no delta complex structure representing this identification since the faces of a $2$-simplex must be $1$-simplices and there are no $1$-simplices in this representation. The idea is to use degenerate $1$-simplices. We can use a simplicial set structure like

$X_0 = \{x\}$

$X_2 = \{A\}$

These are the non-degenerate simplices. 

The face maps 

$d_0(A) = d_1(A) = d_2(A) = s_0(x)$. 

### Collapsing a simplex

Let $X$ be a simplicial set with $x$ a simplex. To construct the simplicial set quotient $X/x$, first introduce a new vertex $x_0$ and replace all instances of $x$, i.e. its faces and degeneracies, with degeneracies of $x_0$. 

Let $x$ be a simplex of dimension $n$ (i.e. $x \in X_n$). For $k\geq 0$ define the non-degenerate face set of dimension $k$ of $x$, inductively, $$F_{x,k} = \bigcup_{i=0}^{k+1}d_i(F_{x,k+1}),\,\,\, (k < n)$$ and $F_{x,n} = \{x\}$, $F_{x,k} = \emptyset$ for $ k > n$. The complete face set of dimension $k$ is defined as $$\bar{F}_{x,k} = F_{x,k} \cup \left( \bigcup_{i=0}^{k-1}s_i(\bar{F}_{x,k-1})\right),\,\,\, (k > 0)$$ and $\bar{F}_{x,0} = F_{x,0}$.

To define the simplicial set $X'=(X/x)$, define the simplex set $(X/x)_k$ as $$X'_k = (X_k - \bar{F}_{x,k})\cup \{s_0^k(x_0)\}$$ where $s_0^k$ is the zeroth degeneracy applied $k$-times. We are just removing all the faces and degeneracies of $x$ in dimension $k$ and explicitly adding $s_0^k(x_0)$ which is the only $k$-dimesnsional degeneracy of $x_0$.

The simplicial maps are defined as follows: for any simplex $y \in X'_k$,
$$d'_i(y) = d_i(y) \,\, \mbox{if} \,\, d_i(y) \notin \bar{F}_{x,k-1}$$
$$d'_i(y) = s_0^{k-1}(x_0)\,\, \mbox{otherwise}$$
$$ $$
$$s'_i(y) = s_i(y)\,\,\mbox{if}\,\,s_i(y)\notin \bar{F}_{x,k+1}$$
$$s'_i(y) = s_i^{k+1}(x_0)\,\,\mbox{otherwise}$$

In other words, if a face or degeneracy of $y$ is an instance of $x$ (i.e. an iterated face or degeneracy of $x$), we make the face or degeneracy of $y$ to be equal to the degeneracy of $x_0$ in the corresponding dimension.

### Betti number computation

The boundary matrix $D(k)$ is a $\mbox{len}(X_{k-1}) \times \mbox{len}(X_k)$ matrix computed in the same way as in the delta complex case. 

Let $X_k^{nd} \subset X_k$ be the subset of non-degenerate simplices of dimension $k$. 

In the betti number computation, let $Z^{\circ}_k =  \mbox{Ker}\,D(k) \cap X_k^{nd}$ be the set of non-degenerate $k$-cycles. Let $B^{\circ}_k = \mbox{Im}\,\left( D(k+1)|_{X_{k+1}^{nd}} \right) = \mbox{Im}\,D(k+1) \cap X_k^{nd}$. The homology group $$H_k(X) = \frac{ Z^{\circ}_k}{B^{\circ}_k}.$$

The betti numbers can be computed as
$$\beta(X)_k = \dim_F\left( \frac{Z^{\circ}_k}{B^{\circ}_k} \right) = \mbox{len}(X_k^{nd}) - \mbox{rank}(D(k)) - \mbox{rank}(D(k+1))$$

## Python implementation

We use a list of sets to write the set of simplices $X_k$. We use a downward multi-tree to encode all the face maps and an upward multi-tree to encode all the degeneracy maps. 

In [19]:
#----Some functions related to tree structure--------------

def depth(A):
	if A == []: return -1
	if type(A) == list:
		return 1 + max(depth(a) for a in A)
	else: return -1

def locate(L,x): 
	out =""
	if x in L: return str(L.index(x))
	else:
		for l in L:
			if isinstance(l,list):
				if isnode(l,x):
					out += str(L.index(l)) + str(locate(L,x))
	return out				

def addressnode(L,address):
	node = L
	for turn in address:
		node = node[int(turn)]
	return node	

def depthcopy(L):
	d = depth(L)
	out = []
	if L == []: return []
	for x in L:
		if type(x) == list: out.append(depthcopy(x))
		else: out.append(str(d) + "-" + str(x))
	return out

def depthcopyreplace(L,A,B):
	d = depth(L)
	out = []
	if L == []: return []
	for x in L: 
		if type(x) == list: out.append(depthcopyreplace(x,A,B))
		else:
			if x in A: out.append(str(B[d]))
			else: out.append(x)
	return out
			


### The sset class

In [20]:
class sset():
	def __init__(self, simplices = [], structuretree = []):
		self.simplices = simplices
		self.structuretree = structuretree
		self.topdim = len(self.simplices)-1

	def simplexdim(self,simplex):
		return max(i for i in range(len(self.simplices)) if simplex in self.simplices[i])
		#return depth(self.simplextree(simplex))	

	def Sigma(self,k):
		return list(self.simplices[k])

	def ndg_Sigma(self,k):
		return [x for x in self.simplices[k] if x[0] not in {"0","1","2"}]

	def ndg_simplices(self,k):
		return [x for x in list(self.simplices[k]) if self.is_degenerate(x) == False]

	def codim1sub(self,i):
		X = sset(self.simplices, self.structuretree[i])
		return X

	def dgtree(self,simplex):
		tree = []
		dim = self.simplexdim(simplex)
		if dim >= self.topdim: return [simplex,[]]
		else: 
			tree = [simplex]
			if simplex[0].isdigit():
				for k in range(int(simplex[0])+1):
					self.simplices[dim+1].add(str(k)+"-"+str(simplex))
					tree.append(self.dgtree(str(k)+"-"+str(simplex)))
			else:
				for k in range(dim+1):
					self.simplices[dim+1].add(str(k)+"-"+str(simplex))
					tree.append(self.dgtree(str(k)+"-"+str(simplex)))
		return tree			
	
	def is_degenerate(self,simplex):
		ans = 1
		for i in range(len(self.simplices)):
			for x in self.simplices[i]:
				if simplex in list(self.simplexdegeneracies(x))[1:]: ans = ans*0
		if ans == 0: return True
		else: return False
		
	def simplexdegeneracies(self,simplex):
		return set(flatten(self.dgtree(simplex)))

	
	def simplexlocate(self,simplex):
		out= ""
		struct = self.structuretree
		if simplex in struct: return str(struct.index(simplex))+"-"
		else:
			for l in struct:
				if isinstance(l,list):
					if isnode(l,simplex):
						out += str(struct.index(l)) + str(self.codim1sub(struct.index(l)).simplexlocate(simplex))
		return out


	def add_alldegeneracies(self):
		for i in range(self.topdim + 1):
			for simplex in self.simplices[i]:
			 	self.dgtree(simplex)

	def addsimplex(self,name,simplexdim):
		while (simplexdim > len(self.simplices)-1):
			self.simplices.append(set())
		
		self.simplices[simplexdim].add(str(name))

	def simplextree(self,simplex):
		L = self.structuretree
		addresses = self.simplexlocate(simplex)
		firstaddress = addresses.split("0-")[0]
		for direction in firstaddress:
			L = L[int(direction)]
		return L
	
	def d(self,simplex,i):
		if i in range(self.simplexdim(simplex)+1):
			return self.simplextree(simplex)[i+1][0]
	
	def delta(self,simplex):
		n = len(self.Sigma(self.simplexdim(simplex)))
		m = len(self.Sigma(self.simplexdim(simplex)-1))
		out = np.zeros(m)
		
		for i in range(self.simplexdim(simplex)+1):
			if self.d(simplex,i)[0] not in {"0","1","2"}:
			#if self.is_degenerate(self.d(simplex,i)) == False:
				out[self.Sigma(self.simplexdim(simplex)-1).index(self.d(simplex,i))] += (-1)**i
		return out

			
	def simplexfaces(self,simplex):
		return set(flatten(self.simplextree(simplex)))

	def simplexboundaries(self,simplex):
		L = []
		for k in range(1,len(self.simplextree(simplex))+1):
			L.append(self.simplextree(simplex)[k][0])
		return L

	
	def simplexcollapse(self,simplex):
		print("\n" + "collapsing simplex " + str(simplex) + "...." + "\n")	
		Q = sset(self.simplices, self.structuretree)
		F = Q.simplexfaces(simplex)
		D = set()
		for x in F:
			D = D.union(Q.simplexdegeneracies(x))
		
		collapse = []
		for i in range(len(self.simplices)):
			collapse.append(set())
		for i in range(len(collapse)):
			for x in self.simplices[i]:
				if x not in F.union(D): collapse[i].add(x)
				else: pass
				
		Q1 = sset(collapse,[])
		new_vertex = str(simplex)+"_0"
		Q1.addsimplex(new_vertex,0)
		new_dgs = sorted(list(Q1.simplexdegeneracies(new_vertex)), key = len)	
		

		collapsetree = depthcopyreplace(self.structuretree, F.union(D) , new_dgs)
		return sset(collapse,collapsetree).clean()
		
	
	def clean(self):
		L = set(flatten(self.structuretree))
		for i in range(len(self.simplices)):
			for x in self.simplices[i]:
				if x not in L:
					self.simplices[i] = self.simplices[i] - {str(x)}
		
		return self	
			
																				
	def boundary_matrix(self,k):
		self.clean()
		if k in range(self.topdim +1):
			if k == 0: return np.zeros(len(self.Sigma(0)))
			else:
				n = len(self.Sigma(k))
				m = len(self.Sigma(k-1))
				A = np.zeros((m,n))
				for j in range(n):
					A[:,j] = self.delta(self.Sigma(k)[j])
				return A
		elif k == self.topdim+1: return np.zeros(1)	
		else: pass

	def betti(self):
		self.clean()

		return [len(self.ndg_Sigma(k)) - np.linalg.matrix_rank(self.boundary_matrix(k)) - np.linalg.matrix_rank(self.boundary_matrix(k+1)) for k in range(self.topdim +1)]


### Test run

In [21]:
X = sset([{"x","y","z"},{"a","b","c"},{"A"}], ["A",["a",["x",[]],["y",[]]],["b",["y",[]],["z",[]]],["c",["x",[]],["z",[]]]])
# X is a triangle (homemorphic to a closed disc, hence contractible)
print(X.simplices)
print(X.structuretree)

print("Betti numbers =", X.betti())

Q = X.simplexcollapse("a")
print(Q.simplices)
print(Q.structuretree)

print("Betti numbers=", Q.betti())

Q1 = Q.simplexcollapse("b")

print(Q1.simplices)
print(Q1.structuretree)
print("Betti numbers=", Q1.betti())

Q2 = Q1.simplexcollapse("c")
#Q_2 is homeomorphic to a sphere

print(Q2.simplices)
print(Q2.structuretree)
print("Betti numbers =", Q2.betti())


[{'y', 'x', 'z'}, {'b', 'a', 'c'}, {'A'}]
['A', ['a', ['x', []], ['y', []]], ['b', ['y', []], ['z', []]], ['c', ['x', []], ['z', []]]]
Betti numbers = [1, 0, 0]

collapsing simplex a....

[{'a_0', 'z'}, {'0-a_0', 'b', 'c'}, {'A'}]
['A', ['0-a_0', ['a_0', []], ['a_0', []]], ['b', ['a_0', []], ['z', []]], ['c', ['a_0', []], ['z', []]]]
Betti numbers= [1, 0, 0]

collapsing simplex b....

[{'b_0'}, {'0-b_0', 'c'}, {'A'}]
['A', ['0-b_0', ['b_0', []], ['b_0', []]], ['0-b_0', ['b_0', []], ['b_0', []]], ['c', ['b_0', []], ['b_0', []]]]
Betti numbers= [1, 0, 0]

collapsing simplex c....

[{'c_0'}, {'0-c_0'}, {'A'}]
['A', ['0-c_0', ['c_0', []], ['c_0', []]], ['0-c_0', ['c_0', []], ['c_0', []]], ['0-c_0', ['c_0', []], ['c_0', []]]]
Betti numbers = [1, 0, 1]


In [22]:
T = sset([{'x'},{'a','b','c'},{'A','B'}],[['A',['a',['x',[]],['x',[]]],['b',['x',[]],['x',[]]],['c',['x',[]],['x',[]]]],['B', ['a',['x',[]],['x',[]]],['b',['x',[]],['x',[]]],['c',['x',[]],['x',[]]]]])
#T is a torus
## Collapsing the 1-simplices 'a' and 'b' produces a space homeomorphic to a sphere
print(T.simplices)
print("Betti numbers = ", T.betti())

Q = T.simplexcollapse('a')
print(Q.simplices)
print("Betti numbers = ", Q.betti())

Q1 = Q.simplexcollapse('b')
print(Q1.simplices)
print("Betti numbers = ", Q1.betti())


[{'x'}, {'b', 'a', 'c'}, {'B', 'A'}]
Betti numbers =  [1, 2, 1]

collapsing simplex a....

[{'a_0'}, {'0-a_0', 'b', 'c'}, {'B', 'A'}]
Betti numbers =  [1, 1, 1]

collapsing simplex b....

[{'b_0'}, {'0-b_0', 'c'}, {'B', 'A'}]
Betti numbers =  [1, 0, 1]


## Reducing a simplicial set via iterative collapses

If $X$ is a cell complex and $A$ is a contractible subcomplex then the quotient map $X \to X/A$ is a homotopy equivalence and hence induces an isomorphism in homology. In computational topology it is an undecidable problem to dtermine if a space is contractible. What one can do instead is check if it is homologically trivial. Then $X\to X/A$ induces an isomorphism in homology. This follows from the existence of a long exact sequence:

$$\cdots \to  H_n(A) \to H_n(X) \to H_n(X/A) \to H_{n-1}(A) \to H_{n-1}(X) \to \cdots \to H_1(X) \to H_1(X/A) \to \widetilde{H}_0(A) \to \widetilde{H}_0(X) \to \widetilde{H}_0(X/A) \to 0$$ where 
$H_0 \simeq \widetilde{H}_0\oplus\mathbb{Z}$.

From this we see that if $A$ is contractible; $H_n(A) = 0 ,\, n >0; \widetilde{H}_0(A) = 0$; then $H_n(X) \simeq H_n(X/A)$ for $n > 0$ and $H_0(X) \simeq H_0(X/A)$. More generally, for a non-contractible space $A$ it may me possible to piece together the homology of $X$ from the homology of $A$ and $X/A$ using the long exact sequence. 

In the context of a simplicial set we can replace $X$ with the quotient $X/x$ where $x$ is a sub simplicial set generated by a simplex $x$ of $X$. If $x$ is has trivial homology (i.e. $H_n^{\Delta}(x) = 0 , n > 0$ and $H_0^{\Delta}(x) = \mathbb{Z}$) then the quotient simplcial map $X \to X/x$ induces isomorphism on homology. In the simplicial set class  add a reduce method that replaces $X$ with $X/x$ iteratively for all homologically trivial simplices $x$. The end result of the reduce function is then a sset with a much smaller representation and having the same betti numbers as the original one.

$$X \mapsto X/x_0 \mapsto (X/x_0)/x_1 \mapsto ((X/x_0)/x_1)/x_2 \mapsto \ldots \mapsto \mbox{red}(X)$$ The collapse operation is a series of quotient reductions where each step produces a homologically equivalent sset.



The TDA flowchart should now consist of the following steps
### Point cloud data $\to$ choice of scale $\to$ simplicial complex $\to$ simplicial set $\to$ reduced sset with minimal presentation